<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/dev/examples/AI_Generated_Narrations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔊 Elevating Trailers with Automated Narration

## Overview
Narration is the heartbeat of trailers, injecting excitement and intrigue into every frame. With VideoDB, OpenAI, and elevenlabs.io at your fingertips, adding narration to trailers becomes a thrilling adventure. This tutorial will guide you through the exhilarating process of seamlessly integrating narration into trailers using these powerful tools. Here’s an example of weaving a thrilling storyline from a reel of unrelated, but valuable cinematic shots:


Let’s try adding an automatically generated voiceover to this silent footage in the style and voice of Sir David Attenborough. We shall do this using the powerful tech provided by [Open AI](https://openai.com), [ElevenLabs](https://elevenlabs.io) and [VideoDB](https://videodb.io)



## Setup

### Installing the required packages

In [ ]:
%pip install openai
%pip install videodb

### API keys
Before proceeding, ensure access to [VideoDB](https://videodb.io), [OpenAI](https://openai.com), and [ElevenLabs](https://elevenlabs.io) API key. If not, sign up for API access on the respective platforms.

> You can get VideoDB's API key from 👉🏼 [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required!** )

In [13]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["ELEVEN_LABS_API_KEY"] = ""
os.environ["VIDEO_DB_API_KEY"] = ""

### ElevenLab's Voice ID 

You will also need ElevenLab's VoiceID of a Voice that you want to use.

For this demo, we will be using David Attenborough's Voice.   
Please add [this](https://elevenlabs.io/app/voice-lab/share/eea2654def1e6c5bda5b4ce8f99f8f2c857b71a15cd6188c27d337206ea98177/s6sJbrmNIsT6M7vdjjES) Voice to Your VoiceLab and copy VoiceID from there.



In [15]:
voiceover_artist_id = "VOICEOVER_ARTIST_ID"


### 🌐 Step 1: Connect to VideoDB
Connect to VideoDB using your API key to establish a session for uploading and manipulating video files.

In [4]:
from videodb import connect

# Connect to VideoDB using your API key
conn = connect()

### 🎥 Step 2: Upload Video

In [5]:
video = conn.upload(url='https://www.youtube.com/watch?v=WQmGwmc-XUY')

### 🔍 Step 3: Analyze Scenes and Generate Scene Descriptions

Start by analyzing the scenes within your Video using VideoDB's scene indexing capabilities. This will provide context for generating the script prompt.

In [6]:
video.index_scenes()

Let's view the description of first scene from the video

In [8]:
scenes = video.get_scenes()
print(f"{scenes[0]['start']} - {scenes[0]['end']}")
print(scenes[0]["response"])

0 - 0.7090416666666666
The image captures a fiery blaze, a dynamic dance of flames in vivid shades of orange, gold, and red. Light flickers intensely, radiance expanding, contracting with the fire's rhythm. No specific source is visible; the fire dominates entirely, filling the frame with energetic movement. The luminosity suggests a fierce heat, powerful enough to demand respect and caution. Each tongue of flame is seemingly alive, almost writhing against a darker, indistinct background. This could be a natural fire or a controlled blaze—there’s no context to indicate its origin. Amidst the searing heat, the flames create a mesmeric, albeit destructive, spectacle.


### 🎤 Step 4: Generate Voiceover Script with ChatGPT
Combine scene descriptions with the script prompt, instructing ChatGPT to create a voiceover script in David Attenborough's style.

This script prompt can be refined and tweaked to generate the most suitable output. Check out these examples to explore more usecases.

In [9]:
# Generate narration script with ChatGPT
import openai

client = openai.OpenAI()

script_prompt = "Craft a dynamic narration script for this trailer, incorporating scene descriptions to enhance storytelling. Ensure that the narration aligns seamlessly with the timestamps provided in the scene index."

full_prompt = script_prompt + "\n\n"
for scene in scenes:
  full_prompt += f"- {scene}\n"

openai_res = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": full_prompt}],
)
voiceover_script = openai_res.choices[0].message.content

# If you have ElevenLab's paid plan remove this
voiceover_script = voiceover_script[:2500]

### 🎤 Step 5: Generate Voiceover Audio with ElevenLabs
Utilize the generated script to synthesize AI-generated voiceover narration in David Attenborough's voice using ElevenLabs API.

In [ ]:
import requests


# Call ElevenLabs API to generate voiceover
url = f"https://api.elevenlabs.io/v1/text-to-speech/{voiceover_artist_id}"
headers = {
    "xi-api-key": os.environ.get("ELEVEN_LABS_API_KEY"),
    "Content-Type": "application/json"
}
payload = {
    "model_id": "eleven_monolingual_v1",
    "text": voiceover_script,
    "voice_settings": {
        "stability": 0.5,
        "similarity_boost": 0.5
    }
}
elevenlabs_res = requests.request("POST", url, json=payload, headers=headers)

print(elevenlabs_res.json())

# Save the audio file
audio_file = "audio.mp3"
CHUNK_SIZE = 1024
with open(audio_file, 'wb') as f:
    for chunk in elevenlabs_res.iter_content(chunk_size=CHUNK_SIZE):
        if chunk:
            f.write(chunk)

### 🎬 Step 6: Add Voiceover to Video with VideoDB

In order to use the voiceover generated above, let's upload the audio file (voiceover) to VideoDB first

In [ ]:
audio = conn.upload(file_path=audio_file)

Finally, add the AI-generated voiceover to the silent footage of the underwater world using VideoDB's [timeline feature](https://docs.videodb.io/version-0-0-3-timeline-and-assets-44).

In [ ]:
from videodb.timeline import Timeline
from videodb.asset import VideoAsset, AudioAsset

timeline = Timeline(conn)

video_asset = VideoAsset(asset_id=video.id, start=0, end=180)

audio_asset1 = AudioAsset(asset_id=audio.id, start=5, end=25.5, disable_other_tracks=False)
audio_asset2 = AudioAsset(asset_id=audio.id, start=35, end=49, disable_other_tracks=False)

# add asset overlay
timeline.add_inline(asset=video_asset)
timeline.add_overlay(start=4, asset=audio_asset1)
timeline.add_overlay(start=35, asset=audio_asset2)


### 🪄 Step 7: Review and Share
Preview the video with the integrated voiceover to ensure it functions correctly.   
Once satisfied, generate a stream of the video and share the link for others to view and enjoy this wholesome creation!


In [15]:
from videodb import play_stream

stream_url = timeline.generate_stream()
play_stream(stream_url)

'https://console.videodb.io/player?url=https://dseetlpshk2tb.cloudfront.net/v3/published/manifests/fea6943a-0ac6-4a13-9309-4df35e41f6f7.m3u8'

## Bonus

In [ ]:
from videodb.asset import ImageAsset
from videodb import MediaType, play_stream

image = conn.upload(url="https://raw.githubusercontent.com/video-db/videodb-cookbook/dev/images/chase_trailer.png", media_type=MediaType.image)

image_asset = ImageAsset(
    asset_id=image.id,
    width=1392,
    height=783,
    x=0,
    y=0,
    duration=13
)

timeline.add_overlay(start=67.5, asset=image_asset)

stream_url = timeline.generate_stream()
play_stream(stream_url)

### 🎉 Conclusion:
Congratulations! You have successfully automated the process of creating custom and personalised voiceovers based on a simple prompt and raw video footage using VideoDB, ChatGPT, and ElevenLabs. By leveraging advanced AI technologies, you can enhance the storytelling and immersive experience of your video content. Experiment with different prompts and scene analysis techniques to further improve the quality and accuracy of the voiceovers. Enjoy creating captivating narratives with AI-powered voiceovers using VideoDB! We're excited to see your creations so you're welcome to share them on our Discord channel [here](https://discord.com/invite/py9P639jGz) 